In [1]:
import pandas as pd
import geopandas
import json
import os
from shapely.geometry import Point
import difflib
from geopy.distance import vincenty
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
inline_rc = dict(mpl.rcParams)

In [2]:
#in_folder = "audits/2018_07/data/"
#out_folder = "audits/2018_11/images/"
in_folder = "../data/"
out_folder = "/tmp"

Fichier des tracés:
    https://opendata.stif.info/explore/dataset/bus_lignes/export/
    
Fichier pour faire la correspondance :
    https://opendata.stif.info/explore/dataset/referentiel-des-lignes-stif/export/

In [3]:
opendata_ref = pd.read_csv("ref_data/referentiel-des-lignes-stif.csv",delimiter=";")
#opendata_ref

In [4]:
opendata_tracks = pd.read_csv("ref_data/bus_lignes.csv",delimiter=";")

opendata_tracks.dropna(inplace=True)
opendata_tracks.drop_duplicates('ID_GroupOfLines',inplace=True)

opendata_tracks = opendata_tracks.merge(opendata_ref, how='inner', on='ID_GroupOfLines')

opendata_tracks = opendata_tracks[['Geo Shape', 'ShortName_Line', 'TransportMode', 'NetworkName', 'ExternalCode_Line' ]]

opendata_tracks = opendata_tracks[opendata_tracks['TransportMode']=='bus']
#opendata_tracks.head()

In [5]:
# on prépare les tracés pour analyse
from functools import partial
import pyproj
from shapely import wkt
from shapely.geometry import shape
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='EPSG:4326'),
    pyproj.Proj(init='EPSG:2154'))

def geojson_to_wkt(row):
    if pd.isna(row['Geo Shape']):
        wkt_ = wkt.loads('POINT (2.482068 49.0327237)')
    else :     
        wkt_ = shape(json.loads(row['Geo Shape']))
    return transform(project, wkt_) # on projette, pour avoir plus tard la longueur en mètres

opendata_tracks['geom'] = opendata_tracks.apply(geojson_to_wkt, axis=1)
opendata_lines_geo = geopandas.GeoDataFrame(opendata_tracks, geometry=opendata_tracks.geom)
#opendata_lines_geo

In [6]:
#opendata_lines_geo.plot()
# TODO : graphique aussi sur OSM pour montrer qu'encore une fois, c'est en périphérie que ça manque

In [7]:
opendata_lines_geo['line_length'] = opendata_lines_geo.length/1000
opendata_lines_geo[['line_length', 'ShortName_Line','NetworkName', 'TransportMode', 'ExternalCode_Line','geom' ]]

print("Nombre de lignes avec tracé : {}".format(len(opendata_lines_geo)))

## TODO : pourcentage par rapport au nombre de lignes gtfs

Nombre de lignes avec tracé : 1709


In [8]:
# TODO regarder les lignes GTFS sans tracés pour voir si on les a dans OSM ?

## OSM

In [9]:
osm_lines = pd.read_csv(os.path.join(in_folder, "osm-transit-extractor_lines.csv"), dtype=str)
osm_lines = osm_lines[osm_lines['mode'] == 'bus']


osm_lines = osm_lines[['line_id', 'name', 'osm:ref:FR:STIF:ExternalCode_Line','code', 'colour', 'operator', 'network','shape']].copy()
#osm_lines.head()

In [10]:
def osm_shape_to_wkt(row):
    if pd.isna(row['shape']):
        wkt_ = wkt.loads('POINT (2.482068 49.0327237)')
    else :
        wkt_ = wkt.loads(row['shape'])
    return transform(project, wkt_) # on projette, pour avoir plus tard la longueur en mètres

osm_lines['geom'] = osm_lines.apply(osm_shape_to_wkt, axis=1)
osm_lines_geo = geopandas.GeoDataFrame(osm_lines, geometry=osm_lines.geom)


In [11]:
#osm_lines_geo.plot()

In [12]:
osm_lines_geo['line_length'] = (osm_lines_geo.length ) /1000
osm_lines_geo_ok = osm_lines_geo[osm_lines_geo['line_length'] >= 3].copy() #on supprime les trop petites lignes, qui doivent être incomplètes
osm_lines_geo_ok[['line_id', 'name', 'network','osm:ref:FR:STIF:ExternalCode_Line', 'line_length']]

print("Nombre de lignes avec tracé : {}".format(len(osm_lines_geo_ok)))

tt = len(osm_lines_geo_ok) * 100.0 / len(osm_lines_geo)
"Pourcentage de lignes OSM qui ont un tracé : {}".format(tt)

Nombre de lignes avec tracé : 875


'Pourcentage de lignes OSM qui ont un tracé : 65.44502617801047'

In [13]:
#osm_lines_geo_ok.groupby(['network'])['line_length'].sum()


In [14]:
osm_lines_geo_ok['line_length'].sum()

26149.317138282866

In [15]:
opendata_lines_geo['line_length'].sum()

44616.0409130414

In [16]:
for c in opendata_lines_geo.columns:
    new_col = "opendata_" + c
    opendata_lines_geo.rename(columns={c: new_col}, inplace=True)
for c in osm_lines_geo_ok.columns:
    new_col = "osm_" + c
    osm_lines_geo_ok.rename(columns={c: new_col}, inplace=True)

In [17]:
lines_geo = opendata_lines_geo.merge(osm_lines_geo_ok, how='inner', right_on='osm_osm:ref:FR:STIF:ExternalCode_Line', left_on='opendata_ExternalCode_Line')
#lines_geo[lines_geo['osm_network']=='Noctilien']

In [18]:
networks_km = pd.DataFrame(lines_geo.groupby(['osm_network'])['osm_line_length', 'opendata_line_length'].sum()).reset_index()

#networks_km

In [19]:
len(networks_km[networks_km['osm_line_length']>=networks_km['opendata_line_length']])
## TODO : à creuser : j'ai plus de lignes où OSM a plus de km que l'opendata O_o 
# ou alors c'est parce qu'on a l'aller et le retour dans OSM et qu'un seul tracé simplifié dans l'opendata ?
# à vérifier avec QGIS

# c'est crédible
# voir par exemple la boucle vers le Centre Commercial Claye
#https://ref-lignes-stif.5apps.com/line.html?osm_relation=7728377

73

In [20]:
osm_better = networks_km[networks_km['osm_line_length']>=networks_km['opendata_line_length']]
osm_better.head()

,osm_network,osm_line_length,opendata_line_length
0,Albatrans,625.926925,290.154553
1,Apolo 7,205.981538,162.675704
3,Athis Cars,79.171622,65.979457
5,Autobus du Fort,18.342590,16.049367
7,BrieBus,16.516605,9.720349
